In [1]:
import h5py
import glob
import time
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from skopt import gp_minimize
from keras_dgl.layers import MultiGraphCNN
from Combined_Multizone_Predictor_0512 import *

In [2]:
# tf.test.is_gpu_available()
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
root_path = "C:/Users/Xiaoshi/Dropbox/9_PhD_Semester_9/01_PhD_Research/11_Optimization/"
rm_param, open_param, win_ids = get_room_parameters(root_path)

In [4]:
# load GCN model
GCN_filename = "C:/Users/Xiaoshi/Dropbox/9_PhD_Semester_9/01_PhD_Research/11_Optimization/models/GCN_model_0316_1300.h5"
GCN_model = load_model(GCN_filename, custom_objects={'MultiGraphCNN': MultiGraphCNN}, compile=False)

# load Outdoor model
Outdoor_filename = "C:/Users/Xiaoshi/Dropbox/9_PhD_Semester_9/01_PhD_Research/11_Optimization/models/Outdoor_model_0801_000085.h5"
Outdoor_model = load_model(Outdoor_filename, compile=False)

# load Indoor model
In_mag_filename = "C:/Users/Xiaoshi/Dropbox/9_PhD_Semester_9/01_PhD_Research/11_Optimization/models/Indoor_model_mag_0801_000280.h5"
In_mag_model = load_model(In_mag_filename, compile=False)

In_hres_filename = "C:/Users/Xiaoshi/Dropbox/9_PhD_Semester_9/01_PhD_Research/11_Optimization/models/Indoor_model_uvwp_0801_000225.h5"
In_hres_model = load_model(In_hres_filename, compile=False)

# load Regularize model
Regularizer_filename = "C:/Users/Xiaoshi/Dropbox/9_PhD_Semester_9/01_PhD_Research/11_Optimization/models/regularizer_gen_model_0801_000058.h5"
Regularizer_model = load_model(Regularizer_filename, compile=False)

In [5]:
# These functions define the optimization problem, it relocate specified windows while keeping their dimension
def define_varibles(rm_param, open_param, win_ids, win_ind_op, res):
    sorter = np.argsort(win_ids)
    row_inds = sorter[np.searchsorted(win_ids, win_ind_op, sorter=sorter)].tolist()
    curr_pos = []
    hv_bnd = []
    for ind in row_inds:
        xl, xh, yl, yh, zl, zh, rm1, rm2 = open_param[ind]
        r1_xl, r1_xh, r1_yl, r1_yh, r1_zl, r1_zh = 0.0, float("inf"), 0.0, float("inf"), 0.0, 3.0
        r2_xl, r2_xh, r2_yl, r2_yh, r2_zl, r2_zh = 0.0, float("inf"), 0.0, float("inf"), 0.0, 3.0
        if rm1 >= 0:
            r1_xl, r1_xh, r1_yl, r1_yh, r1_zl, r1_zh = rm_param[int(rm1)]            
        if rm2 >= 0:
            r2_xl, r2_xh, r2_yl, r2_yh, r2_zl, r2_zh = rm_param[int(rm2)]
        x_lbnd, x_hbnd = max(r1_xl, r2_xl), min(r1_xh, r2_xh)
        y_lbnd, y_hbnd = max(r1_yl, r2_yl), min(r1_yh, r2_yh)
        z_lbnd, z_hbnd = max(r1_zl, r2_zl), min(r1_zh, r2_zh)
        if xl == xh:
            width = abs(yh-yl)
            h_low, h_high = y_lbnd, y_hbnd - width
            h_curr = yl
        else:
            width = abs(xh-xl)
            h_low, h_high = x_lbnd, x_hbnd - width
            h_curr = xl
        height = abs(zh-zl)
        v_low, v_high = z_lbnd, z_hbnd - height
        v_curr = zl
        curr_pos += [h_curr/res, v_curr/res]
        hv_bnd += [(h_low/res, h_high/res), (v_low/res, v_high/res)]
    return np.array(curr_pos).astype(int), np.array(hv_bnd).astype(int)

def update_rm_param(rm_param, open_param, win_ids, win_ind_op, res, x):
    """
    x: a list / 1D vector of windows' int position on a vertical wall in format of [w1_hpos, w1_vpos, w2_hpos, w2_vpos ......]
    """
    row_inds = np.searchsorted(win_ids, win_ind_op).tolist()
    new_open_param = open_param.copy()
    
    for i, ind in enumerate(row_inds):
        hpos, vpos = x[i*2]*res, x[i*2+1]*res
        xl, xh, yl, yh, zl, zh, rm1, rm2 = open_param[ind]
        height = abs(zh-zl)
        zl = vpos
        zh = vpos + height
        if xl == xh:
            width = abs(yh-yl)
            yl, yh = hpos, hpos + width
        else:
            width = abs(xh-xl)
            xl, xh = hpos, hpos + width
        new_open_param[ind] = [xl, xh, yl, yh, zl, zh, rm1, rm2]
    
    new_params = (rm_param, new_open_param)
    return new_params

In [6]:
def airflow_pattern_evaluation(flow_pattern, rm_param, low_thres, up_thres, ext_nxy, ext_nz, res):
    """
    Term A is the summation of undesired velocity potions in 5 rooms. Undesired 
    velocity is defined as less than 0.1m/s (low_thres) or above 0.5m/s (up_thres) 
    and the position should be below 2m in vertical dimension.
    """
    x0_min = min([float(rm_param[i][0]) for i in range(len(rm_param))])
    x1_max = max([float(rm_param[i][1]) for i in range(len(rm_param))])
    y0_min = min([float(rm_param[i][2]) for i in range(len(rm_param))])
    y1_max = max([float(rm_param[i][3]) for i in range(len(rm_param))])
    x_length, y_length= abs(x1_max-x0_min), abs(y1_max-y0_min)
    
    x_s = x0_min - round((ext_nxy-x_length/res)/3.0)*res + res/2.0
    y_s = y0_min - round((ext_nxy-y_length/res)/2.0)*res + res/2.0
    z_s = 0.0 + res/2.0
    
    proportions = []
    for i in range(len(rm_param)):
        rxl, rxh, ryl, ryh, rzl, rzh = rm_param[i]
        ixl, ixh, iyl, iyh, izl, izh = int((rxl+res/2.0-x_s)/res), int((rxh+res/2.0-x_s)/res), \
            int((ryl+res/2.0-y_s)/res), int((ryh+res/2.0-y_s)/res), int((rzl+res/2.0-z_s)/res), int((rzh+res/2.0-z_s)/res)
        rm_i = flow_pattern[ixl:ixh, iyl:iyh, izl:izl+8, :] # position should be below 2m in vertical dimension
        rm_mag = np.sqrt(np.square(rm_i[:,:,:,0])+np.square(rm_i[:,:,:,1])+np.square(rm_i[:,:,:,2]))
        prop = np.sum((rm_mag<low_thres)|(rm_mag>up_thres))/rm_mag.size
        # prop = np.sum((rm_mag<low_thres))/rm_mag.size
        proportions.append(prop.round(6))
    
    term_a = np.sum(proportions).round(6)
    print("rm prop: " + str(proportions) + " | sum Prop: " + str(term_a))
    return term_a

In [7]:
def objective_func(x):
    delta = 0.25
    ext_nxy, ext_nz, nxy, nz = 128, 32, 32, 16
    low_thres = 0.1
    up_thres = 0.5
    new_rm_op_param = update_rm_param(rm_param, open_param, win_ids, win_ind_op, delta, x)
    predicted_flow_pattern = predict_one_case(new_rm_op_param, GCN_model, Outdoor_model, In_mag_model, In_hres_model, Regularizer_model, ext_nxy, ext_nz, nxy, nz, delta)
    loss = airflow_pattern_evaluation(predicted_flow_pattern, rm_param, low_thres, up_thres, ext_nxy, ext_nz, delta)
    return loss

In [8]:
delta = 0.25
ext_nxy, ext_nz, nxy, nz = 128, 32, 32, 16
low_thres = 0.1
up_thres = 0.5

npz_file = root_path + "Result_examples/overall_iteration_1995.npz"
data = np.load(npz_file)
predicted_flow_pattern = data['arr_1']
airflow_pattern_evaluation(predicted_flow_pattern, rm_param, low_thres, up_thres, ext_nxy, ext_nz, delta)

rm prop: [0.78281, 0.696676, 0.901878, 0.604001, 0.86339] | sum Prop: 3.848755


3.848755

In [15]:
# these indices of windows are based on Grasshopper indexing of window openings.
win_ind_op = [3, 5]
delta = 0.25
curr_pos, hv_bnd = define_varibles(rm_param, open_param, win_ids, win_ind_op, delta)
# curr_pos = [[item] for item in curr_pos]
print(list(curr_pos))
print(hv_bnd)
start = time.time()
res = gp_minimize(objective_func,            # the function to minimize
                  hv_bnd,                    # [(-2.0, 2.0)], the bounds on each dimension of x
                  acq_func = "EI",           # the acquisition function
                  n_calls = 50,              # the number of evaluations of f
                  n_initial_points = 5,      # the number of random initialization points
                  n_restarts_optimizer = 5,  
                  acq_optimizer = "lbfgs",
                  noise = "gaussian", 
                  random_state = 42,
                  n_jobs = 5,
                  x0 = list(curr_pos))
print("Case finished optimization in " + str(int(time.time()-start)) +" seconds =========================")

[52, 4, 52, 4]
[[46 56]
 [ 0  7]
 [27 52]
 [ 0  7]]
Start predicting new case ...
The order of room index to predict:  [0 4 1 2 3]
Multizone airflow prediction completed in 31 seconds =========================
rm prop: [0.672907, 0.670533, 0.770962, 0.800265, 0.796246] | sum Prop: 3.710913
Start predicting new case ...
The order of room index to predict:  [0 4 1 2 3]
Multizone airflow prediction completed in 31 seconds =========================
rm prop: [0.682367, 0.458622, 0.720293, 0.277282, 0.768769] | sum Prop: 2.907333
Start predicting new case ...
The order of room index to predict:  [0 4 1 2 3]
Multizone airflow prediction completed in 31 seconds =========================
rm prop: [0.672705, 0.567001, 0.7482, 0.508929, 0.803406] | sum Prop: 3.300241
Start predicting new case ...
The order of room index to predict:  [0 4 1 2 3]
Multizone airflow prediction completed in 31 seconds =========================
rm prop: [0.638486, 0.665166, 0.6875, 0.260251, 0.848684] | sum Prop: 3.100

C:\Users\Xiaoshi\anaconda3\envs\tf-gpu\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Start predicting new case ...
The order of room index to predict:  [0 4 1 2 3]
Multizone airflow prediction completed in 31 seconds =========================
rm prop: [0.657407, 0.237881, 0.737783, 0.201224, 0.592879] | sum Prop: 2.427174
Start predicting new case ...
The order of room index to predict:  [0 4 1 2 3]
Multizone airflow prediction completed in 31 seconds =========================
rm prop: [0.658213, 0.237188, 0.730453, 0.159226, 0.598878] | sum Prop: 2.383958
Start predicting new case ...
The order of room index to predict:  [0 4 1 2 3]
Multizone airflow prediction completed in 31 seconds =========================
rm prop: [0.659823, 0.231129, 0.722351, 0.114253, 0.593073] | sum Prop: 2.320629
Start predicting new case ...
The order of room index to predict:  [0 4 1 2 3]
Multizone airflow prediction completed in 31 seconds =========================
rm prop: [0.673913, 0.425035, 0.678112, 0.226521, 0.722523] | sum Prop: 2.726104
Start predicting new case ...
The order of r

In [13]:
op_param_optimal = update_rm_param(rm_param, open_param, win_ids, win_ind_op, delta, res.x)[1]
optimal_op_file = root_path + "opening_optimal.csv"
np.savetxt(optimal_op_file, op_param_optimal, fmt = "%10.2f", delimiter=",")
print(op_param_optimal)

[[ 7.75  9.5  11.5  11.5   0.    3.    1.    4.  ]
 [ 0.    0.   11.5  13.25  1.75  3.   -4.    4.  ]
 [14.75 14.75  6.75  9.5   0.    1.25 -2.    3.  ]
 [ 9.5  10.5  15.75 15.75  0.75  2.25 -1.    3.  ]
 [14.75 14.75  0.    4.    2.25  3.   -2.    2.  ]
 [ 5.75  5.75  0.    1.    0.    2.    0.    2.  ]
 [ 0.    0.    3.75  5.5   1.    2.5  -4.    0.  ]
 [ 5.75  7.25  6.75  6.75  0.    3.    1.    2.  ]
 [ 9.5   9.5   6.75  9.    1.    2.5   1.    3.  ]]


## Optimization Effectiveness Evaluation 

In [9]:
def iter_get_room_para(root_path, ind):
    room_file_pth = root_path + "design_iteration_" + str(ind) + "/design_"+str(ind)+"_room.csv"
    open_file_pth = root_path + "design_iteration_" + str(ind) + "/design_"+str(ind)+"_open.csv"
    
    # read room data
    rm_file = open(room_file_pth,"r")
    param = list(csv.reader(rm_file, delimiter = ','))[:-1]
    param = np.array(param).astype(float)
    rm_file.close()

    op_file = open(open_file_pth,"r")
    openings = list(csv.reader(op_file, delimiter = ','))
    openings = np.array(openings).astype(float)
    op_file.close()
    
    return param, openings

In [10]:
delta = 0.25
num_sel_win = 4
ext_nxy, ext_nz, nxy, nz = 128, 32, 32, 16
low_thres, up_thres = 0.1, 0.5

root_path = "E:/00_Data_Generator/CFD_result_data/"
npz_path = "C:/Users/Xiaoshi/Desktop/00_Data_Generator/Indoor_Dataset_E/"
target_path = "C:/Users/Xiaoshi/Dropbox/9_PhD_Semester_9/01_PhD_Research/11_Optimization/"

f = open(target_path + "sel_win_ind_list.csv", 'w', newline='')
writer = csv.writer(f)
writer.writerow(["index"] + ["op_id_" + str(i) for i in range(num_sel_win)])
f.close()

f = open(target_path + "loss_values.csv", 'w', newline='')
writer = csv.writer(f)
writer.writerow(["index", "ori_cfd_loss", "ori_pred_loss", "optml_pred_loss"])
f.close()

for i in range(2500, 2700):
    print("Start optimization of iteration [" + str(i) + "] +++++++++++++++++++++++++++++++++++++++++++++++")
    iter_start = time.time()
    rm_param, open_param = iter_get_room_para(root_path, i)
    
    # calculate loss function based on original npz file
    npz_file = npz_path + "overall_iteration_" + str(i) + ".npz"
    npz_data = np.load(npz_file)
    cfd_flow_pattern = npz_data['arr_1']
    ori_cfd_loss = airflow_pattern_evaluation(cfd_flow_pattern, rm_param, low_thres, up_thres, ext_nxy, ext_nz, delta)
    
    # calculate loss function based on original predicted file
    pred_flow_pattern = predict_one_case((rm_param, open_param), GCN_model, Outdoor_model, In_mag_model, In_hres_model, Regularizer_model, ext_nxy, ext_nz, nxy, nz, delta)
    ori_pred_loss = airflow_pattern_evaluation(pred_flow_pattern, rm_param, low_thres, up_thres, ext_nxy, ext_nz, delta)
    
    # these indices of windows are based on Grasshopper indexing of window openings.
    all_win_ids = np.arange(open_param.shape[0]).tolist()
    sel_ind_op = np.sort(np.random.choice(all_win_ids, num_sel_win, False)).tolist()
    
    # remove the variables that have upper bnd equals lower bnd (cannot be changed and will cause problem with optimizer)
    curr_pos, complete_hv_bnd = define_varibles(rm_param, open_param, all_win_ids, sel_ind_op, delta)
    remove_ind, keep_ind = [], []
    for j in range(num_sel_win*2):
        if complete_hv_bnd[j][0] == complete_hv_bnd[j][1]:
            remove_ind.append(j)
        else:
            keep_ind.append(j)
    filtered_ind = remove_ind + keep_ind
    new_pos = curr_pos[remove_ind]
    curr_start_pos = curr_pos[keep_ind]
    hv_bnd = complete_hv_bnd[keep_ind]
    print(new_pos)
    print(hv_bnd)
    
    def cur_objective_func(x):
        # insert the filtered (unchangable) variables into complete variable set to update rm parameter
        arr = np.concatenate((new_pos, x)).astype(int)
        complete_x = arr[np.argsort(filtered_ind)]
        # update room parameter with complete variable set
        new_rm_op_param = update_rm_param(rm_param, open_param, all_win_ids, sel_ind_op, delta, complete_x)
        predicted_flow_pattern = predict_one_case(new_rm_op_param, GCN_model, Outdoor_model, In_mag_model, In_hres_model, Regularizer_model, ext_nxy, ext_nz, nxy, nz, delta)
        loss = airflow_pattern_evaluation(predicted_flow_pattern, rm_param, low_thres, up_thres, ext_nxy, ext_nz, delta)
        return loss
    
    res = gp_minimize(cur_objective_func,        # the function to minimize
                      hv_bnd,                    # [(-2.0, 2.0)], the bounds on each dimension of x
                      acq_func = "EI",           # the acquisition function
                      n_calls = 50,              # the number of evaluations of f
                      n_initial_points = 5,      # the number of random initialization points
                      n_restarts_optimizer = 5,  
                      acq_optimizer = "lbfgs",
                      noise = "gaussian", 
                      random_state = 42,
                      n_jobs = 5)
                      # x0 = list(curr_start_pos))
    
    # calculate loss function based on optimal predicted file
    optml_pred_loss = res.fun
    
    with open(target_path + "loss_values.csv", 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([i, ori_cfd_loss, ori_pred_loss, optml_pred_loss])
    
    with open(target_path + "sel_win_ind_list.csv", 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(np.concatenate(([i], sel_ind_op)).astype(int))
    
    param_folder = target_path + "Optimized_Result/optimized_iteration_" + str(i) + "/"
    if not os.path.exists(param_folder):
        os.makedirs(param_folder)

    original_rm_path = root_path + "design_iteration_" + str(i) + "/design_" + str(i) + "_room.csv"
    rm_file_path = param_folder + "optimized_" + str(i) + "_room.csv"
    op_file_path = param_folder + "optimized_" + str(i) + "_open.csv"

    shutil.copy(original_rm_path, rm_file_path)
    
    arr_final = np.concatenate((new_pos, res.x)).astype(int)
    complete_x_final = arr_final[np.argsort(filtered_ind)]
    op_param_optimal = update_rm_param(rm_param, open_param, all_win_ids, sel_ind_op, delta, complete_x_final)[1]
    np.savetxt(op_file_path, op_param_optimal, fmt = "%10.2f", delimiter=",")
    print("Case finished optimization in " + str(int(time.time()-iter_start)) +" seconds ============================================")
    print("==========================================================================================")

Start optimization of iteration [2500] +++++++++++++++++++++++++++++++++++++++++++++++
rm prop: [0.835648, 0.362787, 0.606607, 0.505445, 0.356897] | sum Prop: 2.667384
Start predicting new case ...
The order of room index to predict:  [0 2 3 1 4]
Multizone airflow prediction completed in 50 seconds =========================
rm prop: [0.65377, 0.381705, 0.337857, 0.156307, 0.124138] | sum Prop: 1.653777
[]
[[27 28]
 [ 0  6]
 [ 0 14]
 [ 0  4]
 [33 48]
 [ 0  4]
 [28 35]
 [ 0 10]]
Start predicting new case ...
The order of room index to predict:  [0 2 3 1 4]
Multizone airflow prediction completed in 28 seconds =========================
rm prop: [0.65377, 0.381705, 0.337857, 0.156307, 0.124138] | sum Prop: 1.653777
Start predicting new case ...
The order of room index to predict:  [0 2 3 1 4]
Multizone airflow prediction completed in 27 seconds =========================
rm prop: [0.619874, 0.261015, 0.285714, 0.396098, 0.120402] | sum Prop: 1.683103
Start predicting new case ...
The order o

Start predicting new case ...
The order of room index to predict:  [0 2 3 1 4]
Multizone airflow prediction completed in 27 seconds =========================
rm prop: [0.598876, 0.310824, 0.314464, 0.326906, 0.141379] | sum Prop: 1.692449
Start predicting new case ...
The order of room index to predict:  [0 2 3 1 4]
Multizone airflow prediction completed in 28 seconds =========================
rm prop: [0.61789, 0.285201, 0.24875, 0.485935, 0.215805] | sum Prop: 1.853581
Start predicting new case ...
The order of room index to predict:  [0 2 3 1 4]
Multizone airflow prediction completed in 27 seconds =========================
rm prop: [0.531746, 0.249042, 0.285893, 0.173321, 0.123276] | sum Prop: 1.363278
Start predicting new case ...
The order of room index to predict:  [0 2 3 1 4]
Multizone airflow prediction completed in 27 seconds =========================
rm prop: [0.660384, 0.236111, 0.343571, 0.220508, 0.110057] | sum Prop: 1.570631
Start predicting new case ...
The order of roo

Start predicting new case ...
The order of room index to predict:  [0 2 1 3 4]
Multizone airflow prediction completed in 26 seconds =========================
rm prop: [0.752273, 0.447343, 0.512175, 0.403571, 0.28853] | sum Prop: 2.403892
Start predicting new case ...
The order of room index to predict:  [0 2 1 3 4]
Multizone airflow prediction completed in 25 seconds =========================
rm prop: [0.761742, 0.209915, 0.459903, 0.300564, 0.443548] | sum Prop: 2.175672
Start predicting new case ...
The order of room index to predict:  [0 2 1 3 4]
Multizone airflow prediction completed in 27 seconds =========================
rm prop: [0.840909, 0.212049, 0.464286, 0.450752, 0.478047] | sum Prop: 2.446043
Start predicting new case ...
The order of room index to predict:  [0 2 1 3 4]
Multizone airflow prediction completed in 27 seconds =========================
rm prop: [0.768182, 0.619782, 0.716883, 0.196241, 0.253584] | sum Prop: 2.554672
Start predicting new case ...
The order of ro

Case finished optimization in 1310 seconds ============================================
Start optimization of iteration [2502] +++++++++++++++++++++++++++++++++++++++++++++++
rm prop: [0.17562, 0.887804, 0.608988, 0.65947, 0.664087] | sum Prop: 2.995969
Start predicting new case ...
The order of room index to predict:  [1 4 2 3 0]
Multizone airflow prediction completed in 23 seconds =========================
rm prop: [0.17562, 0.639452, 0.229081, 0.282576, 0.367647] | sum Prop: 1.694376
[]
[[ 0 23]
 [ 0  4]
 [ 0 17]
 [ 0  5]
 [11 20]
 [ 0  3]
 [ 0  5]
 [ 0 10]]
Start predicting new case ...
The order of room index to predict:  [1 4 2 3 0]
Multizone airflow prediction completed in 25 seconds =========================
rm prop: [0.17562, 0.639452, 0.229081, 0.282576, 0.367647] | sum Prop: 1.694376
Start predicting new case ...
The order of room index to predict:  [1 4 2 3 0]
Multizone airflow prediction completed in 24 seconds =========================
rm prop: [0.121384, 0.615112, 0.1815

Start predicting new case ...
The order of room index to predict:  [1 4 2 3 0]
Multizone airflow prediction completed in 25 seconds =========================
rm prop: [0.174587, 0.613083, 0.116736, 0.109848, 0.337268] | sum Prop: 1.351522
Start predicting new case ...
The order of room index to predict:  [1 4 2 3 0]
Multizone airflow prediction completed in 25 seconds =========================
rm prop: [0.127583, 0.606871, 0.134039, 0.071591, 0.330689] | sum Prop: 1.270773
Start predicting new case ...
The order of room index to predict:  [1 4 2 3 0]
Multizone airflow prediction completed in 25 seconds =========================
rm prop: [0.113636, 0.622591, 0.127841, 0.076894, 0.333978] | sum Prop: 1.27494
Start predicting new case ...
The order of room index to predict:  [1 4 2 3 0]
Multizone airflow prediction completed in 24 seconds =========================
rm prop: [0.06095, 0.618534, 0.192149, 0.077273, 0.328367] | sum Prop: 1.277273
Start predicting new case ...
The order of roo

...

Start predicting new case ...
The order of room index to predict:  [2 0 4 3 1]
Multizone airflow prediction completed in 22 seconds =========================
rm prop: [0.576471, 0.334704, 0.849432, 0.507068, 0.602273] | sum Prop: 2.869948
Start predicting new case ...
The order of room index to predict:  [2 0 4 3 1]
Multizone airflow prediction completed in 22 seconds =========================
rm prop: [0.686949, 0.33347, 0.804924, 0.664807, 0.589134] | sum Prop: 3.079284
Start predicting new case ...
The order of room index to predict:  [2 0 4 3 1]
Multizone airflow prediction completed in 23 seconds =========================
rm prop: [0.431066, 0.29153, 0.840909, 0.448289, 0.60902] | sum Prop: 2.620814
Start predicting new case ...
The order of room index to predict:  [2 0 4 3 1]
Multizone airflow prediction completed in 22 seconds =========================
rm prop: [0.475735, 0.662692, 0.850379, 0.600446, 0.645952] | sum Prop: 3.235204
Start predicting new case ...
The order of room

The order of room index to predict:  [0 4 1 3 2]
Multizone airflow prediction completed in 17 seconds =========================
rm prop: [0.776414, 0.559949, 0.250411, 0.685033, 0.466601] | sum Prop: 2.738408
Start predicting new case ...
The order of room index to predict:  [0 4 1 3 2]
Multizone airflow prediction completed in 17 seconds =========================
rm prop: [0.739583, 0.66284, 0.25699, 0.726974, 0.443122] | sum Prop: 2.829509
Start predicting new case ...
The order of room index to predict:  [0 4 1 3 2]
Multizone airflow prediction completed in 17 seconds =========================
rm prop: [0.767857, 0.559949, 0.344161, 0.676398, 0.378307] | sum Prop: 2.726672
Start predicting new case ...
The order of room index to predict:  [0 4 1 3 2]
Multizone airflow prediction completed in 17 seconds =========================
rm prop: [0.806176, 0.507653, 0.215461, 0.674753, 0.318783] | sum Prop: 2.522826
Start predicting new case ...
The order of room index to predict:  [0 4 1 3 

C:\Users\Xiaoshi\anaconda3\envs\tf-gpu\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Start predicting new case ...
The order of room index to predict:  [0 4 1 3 2]
Multizone airflow prediction completed in 17 seconds =========================
rm prop: [0.819196, 0.444303, 0.209498, 0.687911, 0.260251] | sum Prop: 2.421159
Start predicting new case ...
The order of room index to predict:  [0 4 1 3 2]
Multizone airflow prediction completed in 17 seconds =========================
rm prop: [0.833705, 0.472789, 0.24116, 0.684622, 0.247685] | sum Prop: 2.479961
Start predicting new case ...
The order of room index to predict:  [0 4 1 3 2]
Multizone airflow prediction completed in 17 seconds =========================
rm prop: [0.800967, 0.593112, 0.433183, 0.698191, 0.25] | sum Prop: 2.775453
Start predicting new case ...
The order of room index to predict:  [0 4 1 3 2]
Multizone airflow prediction completed in 17 seconds =========================
rm prop: [0.808036, 0.478741, 0.205592, 0.677632, 0.426587] | sum Prop: 2.596588
Start predicting new case ...
The order of room i

Start predicting new case ...
The order of room index to predict:  [0 1 4 2 3]
Multizone airflow prediction completed in 21 seconds =========================
rm prop: [0.891046, 0.263889, 0.418527, 0.274068, 0.851449] | sum Prop: 2.698979
Start predicting new case ...
The order of room index to predict:  [0 1 4 2 3]
Multizone airflow prediction completed in 21 seconds =========================
rm prop: [0.877329, 0.289187, 0.635324, 0.238548, 0.839372] | sum Prop: 2.87976
Start predicting new case ...
The order of room index to predict:  [0 1 4 2 3]
Multizone airflow prediction completed in 20 seconds =========================
rm prop: [0.848344, 0.302083, 0.439453, 0.16712, 0.810386] | sum Prop: 2.567386
Start predicting new case ...
The order of room index to predict:  [0 1 4 2 3]
Multizone airflow prediction completed in 20 seconds =========================
rm prop: [0.821429, 0.287698, 0.460938, 0.247283, 0.794283] | sum Prop: 2.611631
Start predicting new case ...
The order of roo